In [93]:
import pandas as pd
import numpy as np
import re
import html
from number_parser import parse_ordinal

In [94]:
df = pd.read_csv("DOHMH_New_York_City_Restaurant_Inspection_Results_20231202_z.csv")

In [95]:
df.shape

(209789, 27)

In [96]:
pd.set_option('display.max_columns', None)


In [97]:
df.drop_duplicates()

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Location Point1
0,50106271,SOUR MOUSE,Manhattan,110,DELANCEY STREET,10002.0,6462567220,NaN,01/01/1900,NaN,NaN,NaN,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.718640,-73.988481,103.0,1.0,1800.0,1087565.0,1.004100e+09,MN27,NaN
1,50117820,MODELLO BAR - BARCLAYS CENTER,Brooklyn,620,ATLANTIC AVENUE,11217.0,9174174384,NaN,01/01/1900,NaN,NaN,NaN,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.683447,-73.975691,302.0,35.0,12902.0,3398156.0,3.011180e+09,BK37,NaN
2,50141790,NaN,Manhattan,30,EAST 20 STREET,10003.0,9176670036,NaN,01/01/1900,NaN,NaN,NaN,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.738929,-73.989221,105.0,2.0,5200.0,1016181.0,1.008480e+09,MN13,NaN
3,50134862,FINE AND RAW CHOCOLATE,Brooklyn,70,SCOTT AVENUE,11237.0,6462440734,NaN,01/01/1900,NaN,NaN,NaN,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.709853,-73.922706,301.0,34.0,44900.0,3070597.0,3.029900e+09,BK90,NaN
4,50127238,THE DELUXE PARTY KTV,Queens,3420,LINDEN PL,11354.0,9172957843,NaN,01/01/1900,NaN,NaN,NaN,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.766275,-73.831338,407.0,20.0,86900.0,4112007.0,4.049500e+09,QN22,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209784,50129459,BLUEBERRY,Brooklyn,1849,CONEY ISLAND AVENUE,11230.0,3473719836,Bakery Products/Desserts,08/03/2023,Violations were cited in the following area(s).,02A,Time/Temperature Control for Safety (TCS) food...,Critical,50.0,NaN,NaN,12/01/2023,Cycle Inspection / Initial Inspection,40.611980,-73.962744,314.0,48.0,54200.0,3181708.0,3.067580e+09,BK43,NaN
209785,41444866,PIES-N-THIGHS,Brooklyn,166,SOUTH 4 STREET,11211.0,3475296090,American,08/09/2023,Violations were cited in the following area(s).,04N,Filth flies or food/refuse/sewage associated w...,Critical,18.0,B,08/09/2023,12/01/2023,Cycle Inspection / Re-inspection,40.711289,-73.961300,301.0,34.0,52300.0,3063458.0,3.024460e+09,BK73,NaN
209786,50123805,QK Restaurant Inc.,Queens,4626,KISSENA BLVD,11355.0,7189990806,Chinese,08/01/2022,Violations were cited in the following area(s).,06D,"Food contact surface not properly washed, rins...",Critical,29.0,C,08/01/2022,12/01/2023,Pre-permit (Operational) / Re-inspection,40.750040,-73.818520,407.0,20.0,84500.0,4314742.0,4.051490e+09,QN22,NaN
209787,50056510,GINBO'S HAMBURGER HOUSE,Bronx,118,EAST 170 STREET,10452.0,3479634644,Hamburgers,09/28/2022,Violations were cited in the following area(s).,08A,Establishment is not free of harborage or cond...,Not Critical,21.0,NaN,NaN,12/01/2023,Cycle Inspection / Compliance Inspection,40.839304,-73.915334,204.0,16.0,22102.0,2008084.0,2.028420e+09,BX63,NaN


In [98]:
df["BIN"].value_counts()

4000000.0    1103
1000000.0     448
3000000.0     356
4045999.0     299
5039658.0     264
             ... 
4166382.0       1
3149261.0       1
4049895.0       1
3251759.0       1
1000845.0       1
Name: BIN, Length: 20072, dtype: int64

In [99]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209789 entries, 0 to 209788
Data columns (total 27 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   CAMIS                  209789 non-null  int64  
 1   DBA                    209165 non-null  object 
 2   BORO                   209789 non-null  object 
 3   BUILDING               209481 non-null  object 
 4   STREET                 209783 non-null  object 
 5   ZIPCODE                207093 non-null  float64
 6   PHONE                  209781 non-null  object 
 7   CUISINE DESCRIPTION    207365 non-null  object 
 8   INSPECTION DATE        209789 non-null  object 
 9   ACTION                 207365 non-null  object 
 10  VIOLATION CODE         206208 non-null  object 
 11  VIOLATION DESCRIPTION  206208 non-null  object 
 12  CRITICAL FLAG          209789 non-null  object 
 13  SCORE                  199793 non-null  float64
 14  GRADE                  102961 non-nu

In [100]:
# checking numerical columns in the dataset

numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

df.select_dtypes(include=numerics).shape[1]

# so there are 11 numerical columns in the dataset and rest 16 are non-numerical columns s

11

In [101]:
# checking null values in each column
missing_values_percentage = round(df.isnull().sum().sort_values(ascending=False)/len(df) * 100, 2)
missing_values_percentage

Location Point1          100.00
GRADE DATE                55.09
GRADE                     50.92
SCORE                      4.76
BIN                        2.02
VIOLATION CODE             1.71
VIOLATION DESCRIPTION      1.71
Council District           1.54
Census Tract               1.54
Community Board            1.54
NTA                        1.54
ZIPCODE                    1.29
CUISINE DESCRIPTION        1.16
ACTION                     1.16
INSPECTION TYPE            1.16
DBA                        0.30
BBL                        0.26
BUILDING                   0.15
Latitude                   0.13
Longitude                  0.13
PHONE                      0.00
STREET                     0.00
BORO                       0.00
RECORD DATE                0.00
CRITICAL FLAG              0.00
INSPECTION DATE            0.00
CAMIS                      0.00
dtype: float64

### there are 3 columns in the dataset which have more than 50 percent null values 

In [102]:
df = df.drop(["Location Point1"], axis=1)

## Checking null values in my columns

In [103]:
df.dropna(subset = ["DBA"],inplace=True)

In [104]:
df["BORO"].value_counts()


Manhattan        77486
Brooklyn         55962
Queens           50020
Bronx            18463
Staten Island     7232
0                    2
Name: BORO, dtype: int64

In [105]:
df = df[df["BORO"] != 0]

In [106]:
df.shape[0]

209165

## function to make title of all the values in the column and correct the problems that arise due to title
  

In [107]:
def make_title_and_corrections(df, column):
    df[column] = df[column].str.title()
    
    df[column] = df[column].fillna("")
    
    pattern = r'\b(\d+)(Th|Nd|St|Rd)\b'
    pattern_1 = r'\b([A-Za-z]+)(\'[A-Z])\b'
    corp_pattern = r'\bCorp(?:oration|orations)?(?![.])\b'
    inc_pattern = r'\bInc(?:orporated)?(?![.])\b'

    df[column] = df[column].apply(lambda x : re.sub(pattern, lambda m: m.group(1) + m.group(2).lower(), x))
    df[column] = df[column].apply(lambda x: re.sub(pattern_1,lambda m : m.group(1) + m.group(2).lower(),x ))
    
    df[column] = df[column].str.replace("Llc","LLC")
    df[column] = df[column].str.replace(inc_pattern,"Inc.",regex=True)
    df[column] = df[column].str.replace(corp_pattern,"Corp.",regex=True)
    


In [108]:
make_title_and_corrections(df,"DBA")

## Function to clean street column

In [109]:
def column_tansformation(df,column):
    df[column] = df[column].str.title()
    
    def what(x):
        def is_ordinal(word):
            ordinal_words = ['First', 'Second', 'Third', 'Fourth', 'Fifth', 'Sixth', 'Seventh', 'Eighth', 'Ninth', 'Tenth']
            return word in ordinal_words

        l = x.split()

        for i in range(len(l)):
            if is_ordinal(l[i]):
                parsed_word = parse_ordinal(l[i])
                l[i] = str(parsed_word)

        return " ".join(l)
    
    
    def ordinal(m):
        n = int(m.group())
        if 5 <= n <= 20 :
            suffix = 'th'
        else:
            remainder = n % 10
            if remainder == 1:
                suffix = 'st'
            elif remainder == 2:
                suffix = 'nd'
            elif remainder == 3:
                suffix = 'rd'
            else:
                suffix = 'th'
        output = str(n) + suffix
        return output
    
    
    pattern = r'\b(\d+)(Th|Nd|St|Rd)\b'
    ave_pattern = r'\bAve\.?$|\bAve\b|\bAv\b'
    st_pattern = r'\bSt\.?$\b|\bSt\b'
    blvd_pattern = r'\bBlvd\b'
    number = r'\b\d+\b'
    
    
    df[column] = df[column].apply(lambda x : re.sub(pattern, lambda m: m.group(1) + m.group(2).lower(), str(x)))
    df[column] = df[column].str.replace(ave_pattern, 'Avenue', regex = True)
    df[column] = df[column].str.replace(st_pattern, 'Street', regex = True)
    df[column] = df[column].str.replace(blvd_pattern, 'Boulevard', regex = True)
    df[column] = df[column].apply(what)
    df[column] = df[column].str.replace(re.compile(number), lambda x: ordinal(x), regex=True)
    

    
    
    
column_tansformation(df,"STREET")

In [110]:
df

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,50106271,Sour Mouse,Manhattan,110,Delancey Street,10002.0,6462567220,NaN,01/01/1900,NaN,NaN,NaN,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.718640,-73.988481,103.0,1.0,1800.0,1087565.0,1.004100e+09,MN27
1,50117820,Modello Bar - Barclays Center,Brooklyn,620,Atlantic Avenue,11217.0,9174174384,NaN,01/01/1900,NaN,NaN,NaN,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.683447,-73.975691,302.0,35.0,12902.0,3398156.0,3.011180e+09,BK37
3,50134862,Fine And Raw Chocolate,Brooklyn,70,Scott Avenue,11237.0,6462440734,NaN,01/01/1900,NaN,NaN,NaN,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.709853,-73.922706,301.0,34.0,44900.0,3070597.0,3.029900e+09,BK90
4,50127238,The Deluxe Party Ktv,Queens,3420,Linden Pl,11354.0,9172957843,NaN,01/01/1900,NaN,NaN,NaN,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.766275,-73.831338,407.0,20.0,86900.0,4112007.0,4.049500e+09,QN22
5,50142306,Piquitos De Azucar,Bronx,333,East Gun Hill Road,10467.0,9292169019,NaN,01/01/1900,NaN,NaN,NaN,Not Applicable,NaN,NaN,NaN,12/01/2023,NaN,40.878828,-73.873227,207.0,11.0,43100.0,2018643.0,2.033560e+09,BX43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209784,50129459,Blueberry,Brooklyn,1849,Coney Island Avenue,11230.0,3473719836,Bakery Products/Desserts,08/03/2023,Violations were cited in the following area(s).,02A,Time/Temperature Control for Safety (TCS) food...,Critical,50.0,NaN,NaN,12/01/2023,Cycle Inspection / Initial Inspection,40.611980,-73.962744,314.0,48.0,54200.0,3181708.0,3.067580e+09,BK43
209785,41444866,Pies-N-Thighs,Brooklyn,166,South 4th Street,11211.0,3475296090,American,08/09/2023,Violations were cited in the following area(s).,04N,Filth flies or food/refuse/sewage associated w...,Critical,18.0,B,08/09/2023,12/01/2023,Cycle Inspection / Re-inspection,40.711289,-73.961300,301.0,34.0,52300.0,3063458.0,3.024460e+09,BK73
209786,50123805,Qk Restaurant Inc.,Queens,4626,Kissena Boulevard,11355.0,7189990806,Chinese,08/01/2022,Violations were cited in the following area(s).,06D,"Food contact surface not properly washed, rins...",Critical,29.0,C,08/01/2022,12/01/2023,Pre-permit (Operational) / Re-inspection,40.750040,-73.818520,407.0,20.0,84500.0,4314742.0,4.051490e+09,QN22
209787,50056510,Ginbo's Hamburger House,Bronx,118,East 170th Street,10452.0,3479634644,Hamburgers,09/28/2022,Violations were cited in the following area(s).,08A,Establishment is not free of harborage or cond...,Not Critical,21.0,NaN,NaN,12/01/2023,Cycle Inspection / Compliance Inspection,40.839304,-73.915334,204.0,16.0,22102.0,2008084.0,2.028420e+09,BX63


## Removing .0 from zipcode

In [111]:
df['ZIPCODE'] = df['ZIPCODE'].fillna(0).astype(int)


## Replacing NTA codes with NTA names


In [112]:
# importing scraped Nta_data csv file 
nta_df =  pd.read_csv("Nta_data.csv")

In [113]:
nta_df

,Code,Area
0,BX98,Rikers Island
1,SI99,park-cemetery-etc-Staten Island
2,QN99,park-cemetery-etc-Queens
3,QN98,Airport
4,MN50,Stuyvesant Town-Cooper Village
...,...,...
190,BK35,Stuyvesant Heights
191,BK81,Brownsville
192,BK82,East New York
193,MN03,Central Harlem North-Polo Grounds


In [114]:
# checking total null values in NTA column
df[pd.isna(df["NTA"])].shape[0]


3210

In [115]:
# Filling null with UNKOWN in NTA Column
df["NTA"].fillna("UNKNOWN",inplace=True)

In [116]:
# Making a mapping dict out of the columns in nta_df
code_to_area = dict(zip(nta_df['Code'], nta_df['Area']))


In [117]:
code_to_area

{'BX98': 'Rikers Island',
 'SI99': 'park-cemetery-etc-Staten Island',
 'QN99': 'park-cemetery-etc-Queens',
 'QN98': 'Airport',
 'MN50': 'Stuyvesant Town-Cooper Village',
 'BK09': 'Brooklyn Heights-Cobble Hill',
 'QN41': 'Fresh Meadows-Utopia',
 'BX99': 'park-cemetery-etc-Bronx',
 'BK99': 'park-cemetery-etc-Brooklyn',
 'QN45': 'Douglas Manor-Douglaston-Little Neck',
 'QN47': 'Ft. Totten-Bay Terrace-Clearview',
 'QN62': 'Queensboro Hill',
 'QN44': 'Glen Oaks-Floral Park-New Hyde Park',
 'BK46': 'Ocean Parkway South',
 'QN48': 'Auburndale',
 'BX22': 'North Riverdale-Fieldston-Riverdale',
 'QN42': 'Oakland Gardens',
 'QN43': 'Bellerose',
 'SI32': 'Rossville-Woodrow',
 'SI11': 'Charleston-Richmond Valley-Tottenville',
 'SI25': 'Oakwood-Oakwood Beach',
 'QN60': 'Kew Gardens',
 'BK23': 'West Brighton',
 'QN18': 'Rego Park',
 'BX10': 'Pelham Bay-Country Club-City Island',
 'SI48': 'Arden Heights',
 'SI07': 'Westerleigh',
 'MN40': 'Upper East Side-Carnegie Hill',
 'SI01': "Annadale-Huguenot-Pri

In [118]:
# Replace values in the 'NTA' column with corresponding area names

df["NTA"] = df["NTA"].map(code_to_area)

In [119]:
df.sample(5)

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
89576,50108847,Davidovich Bakery,Manhattan,150,East 3rd Street,10009,2123912870,Bagels/Pretzels,01/25/2022,Violations were cited in the following area(s).,06C,Food not protected from potential source of co...,Critical,12.0,A,01/25/2022,12/01/2023,Pre-permit (Operational) / Re-inspection,40.723489,-73.984941,103.0,2.0,3002.0,1004965.0,1.003980e+09,East Village
54903,41255436,El Porton Bar & Restaurant,Manhattan,3151,Broadway,10027,2126657338,Mexican,02/12/2020,Violations were cited in the following area(s).,02G,Cold food item held above 41º F (smoked fish a...,Critical,40.0,C,02/12/2020,12/01/2023,Cycle Inspection / Re-inspection,40.814315,-73.959300,109.0,7.0,21100.0,1059853.0,1.019930e+09,Morningside Heights
46239,41657014,Piccolo Cafe,Manhattan,313,Amsterdam Avenue,10023,2128730962,Italian,04/19/2018,Violations were cited in the following area(s).,06D,"Food contact surface not properly washed, rins...",Critical,24.0,NaN,NaN,12/01/2023,Cycle Inspection / Initial Inspection,40.780322,-73.980368,107.0,6.0,16100.0,1030096.0,1.011468e+09,Upper West Side
79304,40986939,Wasabi Sushi,Brooklyn,2375,86th Street,11214,7182663500,Japanese,09/27/2023,Violations were cited in the following area(s).,02B,Hot TCS food item not held at or above 140 °F.,Critical,22.0,NaN,NaN,12/01/2023,Cycle Inspection / Initial Inspection,40.599522,-73.989838,311.0,43.0,29800.0,3185325.0,3.068580e+09,Bensonhurst East
206250,50096203,Koronet Pizza,Manhattan,1110,Lexington Avenue,10075,6467670424,Pizza,08/29/2022,Violations were cited in the following area(s).,10F,Non-food contact surface or equipment made of ...,Not Critical,44.0,NaN,NaN,12/01/2023,Pre-permit (Operational) / Initial Inspection,40.773981,-73.959617,108.0,4.0,14000.0,1043211.0,1.014120e+09,Upper East Side-Carnegie Hill


In [120]:
df.to_csv("Cleaned_DOHMH_restaurant_inspection_results.csv", index = False)